In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
!pip install pyvi

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.5/8.5 MB 16.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 993.5/993.5 kB 30.4 MB/s eta 0:00:00


In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
import pickle
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import Embedding, Dense, Dropout, Bidirectional, LSTM, GRU, Input, LayerNormalization, GlobalMaxPooling1D,MaxPooling1D, Conv1D, LayerNormalization, Dropout
from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras import Sequential
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from pyvi import ViUtils
from pyvi import ViTokenizer

#1.Evaluation

In [ ]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import f1_score
from sklearn.metrics import recall_score

In [ ]:
df = pd.read_csv('/content/gdrive/MyDrive/Sentiment Analysis Model/Data/Bộ test.csv')
y_test = pd.read_csv('/content/gdrive/MyDrive/Sentiment Analysis Model/Data/Kiểm thử.csv')
df1 = df['none_emoji_review_text'].to_list()

In [ ]:
with open(r"/content/gdrive/MyDrive/Sentiment Analysis Model/Model/tokenize_data.pkl","rb") as input_file:
  my_tokenizer = pickle.load(input_file)
def preprocess_raw_input(raw_input, tokenizer):
  input_text_pre = list(tf.keras.preprocessing.text.text_to_word_sequence(raw_input))
  input_text_pre = " ".join(input_text_pre)
  input_text_pre_accent = ViTokenizer.tokenize(input_text_pre)
  #print('Text preprocessed: ',input_text_pre_accent)
  tokenized_data_text = tokenizer.texts_to_sequences([input_text_pre_accent])
  vec_data = pad_sequences(tokenized_data_text, padding = 'post', maxlen = 108)#số lượng chiều
  return vec_data
def inference_model(input_feature, model):
  output = model(input_feature).numpy()[0]
  result = output.argmax()
  conf = float(output.max())
  label_dict = {'Tiêu cực':0, 'Trung lập':1, 'Tích cực':2}
  label = list(label_dict.keys())
  return label[int(result)]

def prediction(raw_input, tokenizer, model):
  input_model = preprocess_raw_input(raw_input, my_tokenizer)
  result = inference_model(input_model, model)
  return result


BILSTM_model = load_model('/content/gdrive/MyDrive/Sentiment Analysis Model/Model/model_bilstm_1000.h5')
CNN_model = load_model('/content/gdrive/MyDrive/Sentiment Analysis Model/Model/model_cnn.h5')
Transformer_model = load_model('/content/gdrive/MyDrive/Sentiment Analysis Model/Model/Transformer_model_nh11.h5')

#with open(r"tokenize_data.pkl","rb") as input_file:
 # my_tokenizer = pickle.load(input_file)


In [ ]:
bilstm = []
for i in df1:
  predict = prediction(i, my_tokenizer,BILSTM_model)
  bilstm.append(predict)

In [ ]:
cnn = []
for i in df1:
  predict = prediction(i, my_tokenizer,CNN_model)
  cnn.append(predict)

In [ ]:
trans = []
for i in df1:
  predict = prediction(i, my_tokenizer,Transformer_model)
  trans.append(predict)

In [ ]:
#Điểm mô hình BILSTM
print('Điểm: ', accuracy_score(y_test,bilstm))
print('Điểm: ', recall_score(y_test,bilstm,average= None))
print('Điểm: ', precision_score(y_test,bilstm,average= None))
print('Điểm: ', f1_score(y_test,bilstm,average= None))

Điểm:  0.9587628865979382
Điểm:  [1. 0. 1.]
Điểm:  [0.95348837 0.         0.96296296]
Điểm:  [0.97619048 0.         0.98113208]


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
#Điểm mô hình CNN
print('Điểm: ', accuracy_score(y_test,cnn))
print('Điểm: ', recall_score(y_test,cnn,average= None))
print('Điểm: ', precision_score(y_test,cnn,average= None))
print('Điểm: ', f1_score(y_test,cnn,average= None))

Điểm:  0.9381443298969072
Điểm:  [1.         0.         0.96153846]
Điểm:  [0.95348837 0.         0.92592593]
Điểm:  [0.97619048 0.         0.94339623]


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
#Điểm mô hình Transformer
print('Điểm: ', accuracy_score(y_test,trans))
print('Điểm: ', recall_score(y_test,trans,average= None))
print('Điểm: ', precision_score(y_test,trans,average= None))
print('Điểm: ', f1_score(y_test,trans,average= None))

Điểm:  0.9381443298969072
Điểm:  [0.97560976 0.         0.98076923]
Điểm:  [0.97560976 0.         0.91071429]
Điểm:  [0.97560976 0.         0.94444444]


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


#2.Deploying

In [ ]:
from keras.models import load_model
with open(r"/content/gdrive/MyDrive/Sentiment Analysis Model/Model/tokenize_data.pkl","rb") as input_file:
  tokenize_data = pickle.load(input_file)
def preprocess_raw_input(raw_input, tokenizer):
  input_text_pre = list(tf.keras.preprocessing.text.text_to_word_sequence(raw_input))
  input_text_pre = " ".join(input_text_pre)
  input_text_pre_accent = ViTokenizer.tokenize(input_text_pre)
  print('Text preprocessed: ',input_text_pre_accent)
  tokenized_data_text = tokenizer.texts_to_sequences([input_text_pre_accent])
  vec_data = pad_sequences(tokenized_data_text, padding = 'post', maxlen = 108)
  return vec_data
def inference_model(input_feature, model):
  output = model(input_feature).numpy()[0]
  result = output.argmax()
  conf = float(output.max())
  label_dict = {'Tiêu cực':0, 'Trung lập':1, 'Tích cực':2}
  label = list(label_dict.keys())
  return label[int(result)], conf
def prediction(raw_input, tokenizer, model):
  input_model = preprocess_raw_input(raw_input, tokenize_data)
  result, conf = inference_model(input_model, model)
  return result, conf

#my_model = generate_model()
my_model = load_model('/content/gdrive/MyDrive/Sentiment Analysis Model/Model/model_bilstm_1000.h5')

with open(r"/content/gdrive/MyDrive/Sentiment Analysis Model/Model/tokenize_data.pkl","rb") as input_file:
  my_tokenizer = pickle.load(input_file)

print(prediction('Nhân viên phục vụ tốt', my_tokenizer,my_model))

Text preprocessed:  nhân_viên phục_vụ tốt
('Tích cực', 0.8908493518829346)


In [ ]:
while(True):
  text = input("Enter Comment: ")
  if text == 'end':
        break
  else:
    print(prediction(text,my_tokenizer,my_model)[0]+"\n")

Enter Comment: Thái độ nhân viên cực kì tệ
Text preprocessed:  thái_độ nhân_viên cực_kì tệ
Tiêu cực

Enter Comment: Ngân hàng rất ok
Text preprocessed:  ngân_hàng rất ok
Tích cực

Enter Comment: Tôi không thích đi đến ngân hàng này nữa
Text preprocessed:  tôi không thích đi đến ngân_hàng này nữa
Tiêu cực

Enter Comment: Nhân viên thì chậm chạp, dịch vụ quá tệ
Text preprocessed:  nhân_viên thì chậm_chạp dịch_vụ quá tệ
Tiêu cực

Enter Comment: end
